In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout, Dense, LeakyReLU, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam as LegacyAdam


In [2]:
# Height and width refer to the size of the image
# Channels refers to the amount of color channels (red, green, blue)

image_dimensions = {'height':256, 'width':256, 'channels':3}

In [3]:
class Meso4:
    def __init__(self, learning_rate=0.001):
        self.model = self.init_model()
        optimizer = LegacyAdam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer,
                           loss='mean_squared_error',
                           metrics=['accuracy'])

    def init_model(self): 
        x = Input(shape=(256, 256, 3))
        
        x1 = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = Conv2D(8, (5, 5), padding='same', activation='relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)
        
        x3 = Conv2D(16, (5, 5), padding='same', activation='relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = Conv2D(16, (5, 5), padding='same', activation='relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation='sigmoid')(y)

        return Model(inputs=x, outputs=y)

# Create the model architecture
meso = Meso4()

# Load the pre-trained weights
meso.model.load_weights('/Users/taherpanbiharwala/Desktop/Deepfake_Detection/Deepfake-detection/weights/Meso4_DF.h5')


2024-06-25 03:59:45.106525: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-06-25 03:59:45.106545: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-06-25 03:59:45.106549: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-06-25 03:59:45.106752: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-25 03:59:45.106771: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Unfreeze some layers for fine-tuning
for layer in meso.model.layers[-4:]:
    layer.trainable = True

In [5]:
# Prepare your dataset
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/Users/taherpanbiharwala/Desktop/Deepfake_Detection/Dataset/Train',
    target_size=(256, 256),
    batch_size=4,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/Users/taherpanbiharwala/Desktop/Deepfake_Detection/Dataset/Validation',
    target_size=(256, 256),
    batch_size=4,
    class_mode='binary',
    subset='validation')

Found 19202 images belonging to 2 classes.
Found 2200 images belonging to 2 classes.


In [6]:
# Compile the model
meso.model.compile(optimizer=LegacyAdam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
# Define callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint('fine_tuned_meso4.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

# Fine-tune the model
history = meso.model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stop])

Epoch 1/50


2024-06-25 03:59:45.999660: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4801/4801 [==============================] - ETA: 0s - loss: 0.6359 - accuracy: 0.6763

2024-06-25 04:02:00.778513: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4801/4801 [==============================] - 150s 31ms/step - loss: 0.6359 - accuracy: 0.6763 - val_loss: 0.4871 - val_accuracy: 0.7955
Epoch 2/50
   4/4801 [..............................] - ETA: 1:54 - loss: 0.7961 - accuracy: 0.6875

/Users/taherpanbiharwala/anaconda3/envs/project/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4801/4801 [==============================] - 148s 31ms/step - loss: 0.4969 - accuracy: 0.7557 - val_loss: 0.4526 - val_accuracy: 0.8059
Epoch 3/50
4801/4801 [==============================] - 145s 30ms/step - loss: 0.4668 - accuracy: 0.7735 - val_loss: 0.3943 - val_accuracy: 0.8573
Epoch 4/50
4801/4801 [==============================] - 149s 31ms/step - loss: 0.4439 - accuracy: 0.7850 - val_loss: 0.4119 - val_accuracy: 0.8355
Epoch 5/50
4801/4801 [==============================] - 149s 31ms/step - loss: 0.4210 - accuracy: 0.8011 - val_loss: 0.4068 - val_accuracy: 0.8414
Epoch 6/50
4801/4801 [==============================] - 149s 31ms/step - loss: 0.3950 - accuracy: 0.8198 - val_loss: 0.3680 - val_accuracy: 0.8600
Epoch 7/50
4801/4801 [==============================] - 150s 31ms/step - loss: 0.3694 - accuracy: 0.8329 - val_loss: 0.3958 - val_accuracy: 0.8068
Epoch 8/50
4801/4801 [==============================] - 151s 31ms/step - loss: 0.3427 - accuracy: 0.8473 - val_loss: 0.5156 - val

In [8]:
# Evaluate the model
loss, accuracy = meso.model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

550/550 [==============================] - 16s 30ms/step - loss: 0.2552 - accuracy: 0.8900
Validation Loss: 0.2552040219306946
Validation Accuracy: 0.8899999856948853
